In [1]:
from linear import *
from convolutional import *
from activations import *
from loss import CategoricalCrossEntropyLoss
import mnist

np.random.seed(0)

In [2]:
class MnistConv:
    def __init__(self, lr = 0.01) -> None:
        self.layers = [
            ConvLayer(1, 32, 5, activaton=ReLu()),
            MaxPool(2),
            Flatten(),
            Linear(32*12**2, 64, Sigmoid()),
            Linear(64, 10, None)
        ]
        self.lr = lr
        self.res = None
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        self.res = x
        return x
    
    def backward(self, expected_output):
        loss = CategoricalCrossEntropyLoss()
        loss_val = loss(self.res, expected_output)
        err = loss.derivatives()
        for layer in reversed(self.layers):
            err = layer.backward(err, self.lr)
        return loss_val

In [3]:
x_train, t_train, x_test, t_test = mnist.load()
model = MnistConv()

In [4]:
EPOCHES = 3

for epoch in range(EPOCHES):
    print(f"epoch {epoch}/{EPOCHES}")
    loss_val = 0
    for i, (img, label) in enumerate(zip(x_train[:10000], t_train[:10000])):

        res = model.forward(img.reshape((1, 1, 28, 28))/255.0)
        loss_val += model.backward(label)
        if i % 100 == 0 and i > 0:
            print(f"epoch: {epoch}.{i/100}, loss: {loss_val/100}")
            loss_val = 0




epoch 0/3


/home/mpeshkov/Documents/backup/NeuralNetwork/loss.py:3: RuntimeWarning: overflow encountered in exp
  e = np.exp(tensor)
/home/mpeshkov/Documents/backup/NeuralNetwork/loss.py:4: RuntimeWarning: invalid value encountered in divide
  return e/np.sum(e, axis= 1, keepdims=True)
/home/mpeshkov/Documents/backup/NeuralNetwork/loss.py:14: RuntimeWarning: divide by zero encountered in log
  return np.mean(-np.log(self.vals[:, label]))


epoch: 0.1.0, loss: nan


KeyboardInterrupt: 

In [ ]:
results = np.zeros((3, 10))
for (img, label) in zip(x_train[:1000], t_train[:1000]):
    results[0][label] += 1
    r = model.forward(img.reshape((1, 1, 28, 28))/255.0)
    if np.argmax(r, axis = 1) == label:
        results[1][label] +=1
    results[2][np.argmax(r, axis = 1)] += 1

print(results[1])
print(results[0])
print(results[2])

[ 83. 111.  73.  71.  84.  49.  83. 103.  66.  82.]
[ 97. 116.  99.  93. 105.  92.  94. 117.  87. 100.]
[ 89. 135.  87. 100.  97.  70. 111. 123.  80. 108.]


In [ ]:
np.sum(results[1])/np.sum(results[0])

0.805

In [ ]:
results[1]/results[0]

array([0.8556701 , 0.95689655, 0.73737374, 0.76344086, 0.8       ,
       0.5326087 , 0.88297872, 0.88034188, 0.75862069, 0.82      ])